In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
shoes = pd.read_csv('~/Documents/GitHub/DSBA-webscraping/analysis/output.csv')  

for col in shoes:
    shoes[col] = shoes[col].str.strip().str.replace(u'\xa0', ' ')

shoes['regular_price'] = shoes['regular_price'].str.replace(' ', '').astype('float64')
shoes['special_price'] = shoes['special_price'].str.replace(' ', '').astype('float64')
shoes['shoe_type'] = shoes['first_name'].str.split(' ', n=1, expand=True)[0]

shoes = shoes[['first_name', 'second_name', 'shoe_type', 'regular_price', 'special_price']]

shoes[:5]

,first_name,second_name,shoe_type,regular_price,special_price
0,Sandały EVA MINGE,EM-44-07-000811 121,Sandały,399.0,299.0
1,Sandały NESSI,22179 Biały,Sandały,339.0,NaN
2,Sandały BALDACCINI,1697000 Chic Złoto,Sandały,349.0,279.0
3,Sneakersy ROBERTO,3034 Złoto Lico,Sneakersy,449.0,309.0
4,Klapki EVA MINGE,EM-41-09-001139 101,Klapki,389.0,289.0


In [34]:
def get_perc(prop, n=2):
    return str(round(prop*100,n)) + '%'

summary = pd.DataFrame({
    'Shoes count': shoes['first_name'].count(),
    'Shoes on promo': shoes['special_price'].notna().sum(),
    'Avg. regular price': round(shoes['regular_price'].mean(), 2),
    'Avg. regular price if discounted': round(shoes['regular_price'].where(shoes['special_price'].notna()).mean(), 2),
    'Avg. promo price': round(shoes['special_price'].mean(), 2)
}, index=[0])

summary['Shoes discounted (%)'] = (summary['Shoes on promo'] / summary['Shoes count']).apply(get_perc, n=1)
summary['Avg. discount (%)'] = (summary['Avg. promo price'] / summary['Avg. regular price if discounted'] - 1).apply(get_perc, n=1)

summary

,Shoes count,Shoes on promo,Avg. regular price,Avg. regular price if discounted,Avg. promo price,Shoes discounted (%),Avg. discount (%)
0,46304,36332,448.67,478.65,319.72,78.5%,-33.2%


In [60]:
shoes.pivot_table(
    index='shoe_type', 
    values=['first_name', 'regular_price', 'special_price'],
    aggfunc={
        'first_name': 'count',
        'special_price': lambda x: x.notna().sum(),
        'regular_price': np.mean,
    })

,first_name,regular_price,special_price
shoe_type,,,
Baleriny,834,330.996823,615
Botki,6390,596.987773,5863
Buty,2700,484.133852,2260
Cholewka,9,319.000000,9
EVA,1,599.000000,0
Espadryle,2323,360.186659,1663
Glany,79,738.746835,70
Japonki,1295,228.818069,987
Kalosze,219,396.588584,195
